In [8]:
import pandas as pd
import functions as f

### Getting data from website (residentevildatabase.com)

In [9]:
df = f.get_all_personas()

### Transforming data to transacional dataset

In [4]:
def sort_names( row ):
    'Method to drop same relationship, i.e., persona_1 -> persona_2, persona_2 -> persona_1'
    names = [ row["persona_1"], row["persona_2"] ]
    names.sort()
    names = ", ".join(names)
    return names

In [5]:
df_analysis = pd.merge( left=df, right=df,
                        how="left", on="apparition",
                        suffixes=("_1", "_2") )

df_analysis = df_analysis[ df_analysis["persona_1"] != df_analysis["persona_2"] ]

df_analysis["names"] = df_analysis.apply(sort_names, axis=1)
df_analysis = df_analysis.drop_duplicates(subset=["names", "apparition"])

df_analysis = df_analysis[["persona_1", "persona_2", "apparition"]]

df_analysis.head()

,persona_1,persona_2,apparitions
1,ada-wong,annette-birkin,Biohazard - Resident Evil 2
2,ada-wong,ben-bertolucci,Biohazard - Resident Evil 2
3,ada-wong,brian-irons,Biohazard - Resident Evil 2
4,ada-wong,claire-redfield,Biohazard - Resident Evil 2
5,ada-wong,hunk,Biohazard - Resident Evil 2


In [6]:
df_analysis.to_excel("../transational_data.xlsx")